# Proceso de ETL:

## 1. Extracción

En esta etapa, se importan las bibliotecas necesarias y se cargan los conjuntos de datos desde archivos Excel. Los archivos fueron descargados desde la página [Buenos Aires Data](https://data.buenosaires.gob.ar/dataset/victimas-siniestros-viales), específicamente los recursos de lesiones y homicidios.


### Importación de Librerías
Se importan las bibliotecas necesarias para llevar a cabo el proceso ETL.

In [1]:
import pandas as pd
import numpy as np

### Lectura de Archivos:

En esta parte del proceso ETL, se lleva a cabo la lectura de los conjuntos de datos desde archivos Excel. Los archivos se ubican en la ruta './datasets/Originales' y contienen información sobre homicidios y lesiones relacionadas con accidentes viales en la ciudad de Buenos Aires.

In [2]:
homicidios_hechos= pd.read_excel('../datasets/Originales/homicidios.xlsx',sheet_name = 'HECHOS',na_values=['SD', 'sd'])
homicidios_victimas= pd.read_excel('../datasets/Originales/homicidios.xlsx',sheet_name = 'VICTIMAS',na_values=['SD', 'sd'])
lesiones_hechos= pd.read_excel('../datasets/Originales/lesiones.xlsx' , sheet_name = 'HECHOS',na_values=['SD', 'sd'])
lesiones_victimas= pd.read_excel('../datasets/Originales/lesiones.xlsx' , sheet_name = 'VICTIMAS',na_values=['SD', 'sd'])

## 2. Transformación:

En esta etapa, se realiza la manipulación y preparación de los datos para su posterior análisis. Las transformaciones que se llevarán a cabo son las siguientes:

- **Eliminación de columnas:** Se quitarán aquellas columnas que no se utilizarán en este proyecto.
- **Manejo de valores nulos:** Los valores 'SD' y 'sd' se han marcado como valores nulos durante la carga de datos.
- **Cambio de tipo de dato:** Se llevará a cabo la conversión de los datos al tipo más conveniente para el análisis.
- **Renombrar columnas:** Se realizará el cambio del nombre de las columnas según sea necesario.

### DataFrame homicidios_hechos

In [3]:
homicidios_hechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7.0,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0.0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,NaN
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5.0,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


#### Eliminación de columnas

La columna FECHA será conservada, mientras que las columnas AAAA, DD, y MM, que hacen referencia a información de la fecha, serán eliminadas.

In [4]:
homicidios_hechos.drop(columns=['AAAA','DD','MM'],inplace=True)

La columna HH, que representa la hora entera, será conservada, mientras que la columna HORA, que incluye los minutos, será eliminada.

In [5]:
homicidios_hechos.drop(columns=['HORA'],inplace=True)

Con el propósito de simplificar la representación de la ubicación del hecho, se ha decidido utilizar las columnas `pos X` y `pos Y`, que hacen referencia a la latitud y longitud respectivamente. Por lo tanto, se procederá a eliminar las siguientes columnas:

- `LUGAR_DEL_HECHO`
- `Calle`
- `Altura`
- `Cruce`
- `Dirección Normalizada`
- `XY(CABA)`



In [6]:
homicidios_hechos.drop(columns=['LUGAR_DEL_HECHO','Calle','Altura','Cruce','Dirección Normalizada','XY (CABA)'],inplace=True)

#### Valores nulos

Calcular el porcentaje de valores nulos por columnas

In [7]:
# Imprimir el porcentaje de los valores nulos
print('Porcentaje valores nulos')
print('-'*24)
print((homicidios_hechos.isnull().sum()[homicidios_hechos.isnull().sum() > 0] / len(homicidios_hechos)) * 100)

Porcentaje valores nulos
------------------------
HH         0.143678
VICTIMA    1.293103
ACUSADO    3.304598
dtype: float64


El porcentaje de valores nulos en el conjunto de datos es bajo, afectando solo a 3 columnas. Sin embargo, se identificaron irregularidades en la columna de "Comuna", donde la presencia de "Comuna 0" indica una inconsistencia, ya que las comunas deben estar en el rango de 1 a 15. Estos casos también serán tratados como valores nulos. Además, se observaron puntos en las columnas "Pos X" y "Pos Y", y se decidirá reemplazarlos por valores nulos para mantener la consistencia en la representación de coordenadas geográficas.

In [20]:
# Reemplazo de la Comuna 0 por Valores Nulos
homicidios_hechos.loc[homicidios_hechos['COMUNA'] == 0, 'COMUNA'] = np.nan

#Reemplazo de los . por valores nulos de las columnas pos x y pos y

homicidios_hechos.loc[homicidios_hechos['pos x'] == '.' ,'pos x'] = np.nan
homicidios_hechos.loc[homicidios_hechos['pos y'] == '.' ,'pos y'] = np.nan

In [21]:
# Imprimir el porcentaje de los valores nulos
print('Porcentaje valores nulos')
print('-'*24)
print((homicidios_hechos.isnull().sum()[homicidios_hechos.isnull().sum() > 0] / len(homicidios_hechos)) * 100)

Porcentaje valores nulos
------------------------
HH         0.143678
COMUNA     0.287356
pos x      1.724138
pos y      1.724138
VICTIMA    1.293103
ACUSADO    3.304598
dtype: float64


### Cambio de Tipos de Datos

Las columnas 'HH', 'TIPO_DE_CALLE', 'COMUNA' y 'ACUSADO' serán transformadas al tipo de dato categórico.

In [23]:
# cambio a formato categórico
homicidios_hechos['HH'] = homicidios_hechos['HH'].astype('category')
homicidios_hechos['TIPO_DE_CALLE'] = homicidios_hechos['TIPO_DE_CALLE'].astype('category')
homicidios_hechos['COMUNA'] = homicidios_hechos['COMUNA'].astype('category')
homicidios_hechos['ACUSADO'] = homicidios_hechos['ACUSADO'].astype('category')

Las columnas 'pos X' y 'pos Y' serán transformadas al tipo de dato flotante. 

In [26]:
# cambio a flotante los valores de pos x y pos y
homicidios_hechos['pos x'] = homicidios_hechos['pos x'].astype('float')
homicidios_hechos['pos y'] = homicidios_hechos['pos y'].astype('float')

### Renombrar columnas

Todos los nombres de las columnas serán convertidos a minúsculas. 

In [33]:
homicidios_hechos.columns = homicidios_hechos.columns.str.lower()

Renombrar las columnas de 'id', 'pos x' y 'pos y'

In [35]:
# Renombrar columnas de ID, Pos X y Pos Y
homicidios_hechos.rename(columns={'id': 'id_hecho', 'pos x': 'longitud', 'pos y': 'latitud'}, inplace=True)

Las columnas 'tipo_de_calle', 'participantes', 'victima' y 'acusado' transformé el contenido de sus filas en minúsculas.  

In [38]:
# Cambiar el contenido de las filas a minúsculas en columnas específicas
columnas_a_minusc = ['tipo_de_calle', 'participantes', 'victima', 'acusado']
homicidios_hechos[columnas_a_minusc] = homicidios_hechos[columnas_a_minusc].apply(lambda x: x.astype(str).str.lower())

### DataFrame homicidios_victimas

In [40]:
homicidios_victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19.0,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70.0,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30.0,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18.0,NaN
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29.0,2016-02-01 00:00:00


#### Eliminación de columnas

La columna FECHA será conservada, mientras que las columnas AAAA, DD, y MM, que hacen referencia a información de la fecha, serán eliminadas.

In [41]:
#Eliminación de columnas
homicidios_victimas.drop(columns=['AAAA','MM','DD','FECHA_FALLECIMIENTO'],inplace=True)

#### Valores nulos
Calcular el porcentaje de valores nulos por columnas

In [44]:
# Imprimir el porcentaje de los valores nulos
print('Porcentaje valores nulos')
print('-'*24)
print((homicidios_victimas.isnull().sum()[homicidios_victimas.isnull().sum() > 0] / len(homicidios_victimas)) * 100)

Porcentaje valores nulos
------------------------
ROL        1.534170
VICTIMA    1.255230
SEXO       0.836820
EDAD       7.391911
dtype: float64


### Cambio de Tipos de Datos

Conversión de las columnas 'ROL', 'VICTIMA' y 'SEXO' a tipo categórico

In [47]:
# cambio a formato categórico
homicidios_victimas['ROL'] = homicidios_victimas['ROL'].astype('category')
homicidios_victimas['VICTIMA'] = homicidios_victimas['VICTIMA'].astype('category')
homicidios_victimas['SEXO'] = homicidios_victimas['SEXO'].astype('category')


### Renombrar columnas

Todos los nombres de las columnas serán convertidos a minúsculas. 

In [51]:
homicidios_victimas.columns = homicidios_victimas.columns.str.lower()

Las columnas 'rol', 'victima', 'sexo' y 'acusado' transformé el contenido de sus filas en minúsculas.

In [53]:
# Cambiar el contenido de las filas a minúsculas en columnas específicas
columnas_a_minusc = ['rol', 'victima', 'sexo']
homicidios_victimas[columnas_a_minusc] = homicidios_victimas[columnas_a_minusc].apply(lambda x: x.astype(str).str.lower())

Cambo de la etiqueta 'pasajero_acompañante' por 'pasajero' en la columna rol.

In [59]:
#Cambio de etiqueta
homicidios_victimas['rol'].replace('pasajero_acompañante', 'pasajero', inplace=True)


### 3. Carga:

En esta etapa, no se proporciona un código explícito de carga, pero si fuera necesario guardar los datos transformados en una base de datos u otro formato, esta sería la fase en la que se realizaría esa operación.
